In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
#  ! kaggle competitions download -c 'name-of-competition'

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition # to download the data from kaggle api command
#Create a seperate directory for different classes
!mkdir /content/Test
!mkdir /content/Train
!mkdir /content/Train/dog
!mkdir /content/Train/cat
#Unzipping the data
!unzip train.zip
!unzip test.zip
#Copying the data to required folders


In [3]:
import os,shutil
for filename in os.listdir("/content/train"):
  # print(filename)
  if "cat" in filename:
    shutil.copy(os.path.join("/content/train",filename),"/content/Train/cat")
  else :
    shutil.copy(os.path.join("/content/train",filename),"/content/Train/dog")

for filename in os.listdir("/content/test"):
    shutil.copy(os.path.join("/content/test",filename),"/content/Test")

In [13]:
# # Importing the important libraries
# from keras.applications.resnet50 import ResNet50
# from keras.layers import Dense, GlobalAveragePooling2D
# from keras.models import Model
# from keras.optimizers import SGD
# from keras.preprocessing.image import ImageDataGenerator

# # Download the architecture of ResNet50 with ImageNet weights
# base_model = ResNet50(include_top=False, weights='imagenet')

# # Taking the output of the last convolution block in ResNet50
# x = base_model.output

# # Adding a Global Average Pooling layer
# x = GlobalAveragePooling2D()(x)

# # Adding a fully connected layer having 1024 neurons
# x = Dense(1024, activation='relu')(x)

# # Adding a fully connected layer having 2 neurons which will
# # give the probability of image having either dog or cat
# predictions = Dense(2, activation='softmax')(x)

# # Model to be trained
# model = Model(inputs=base_model.input, outputs=predictions)

# # Training only top layers i.e. the layers which we have added in the end
# for layer in base_model.layers:
#     layer.trainable = False

# # Compiling the model
# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])


In [21]:
import tensorflow as tf
from keras.applications import ResNet50
from keras import Sequential
model=Sequential()
model.add(ResNet50(include_top=False, pooling='max', weights='imagenet'))
model.add(Dense(2, activation='softmax'))
# ResNet-50 model is already trained, should not be trained
model.layers[0].trainable = True

In [24]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 23,538,690
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split
batch_size=32
train_generator = train_datagen.flow_from_directory(
    "/content/Train",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') 
# set as training data

validation_generator = train_datagen.flow_from_directory(
    "/content/Train", # same directory as training data
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') 
# set as validation data

# Training the model for 5 epochs
nb_epochs=15
batch_size=32
model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

# # We will try to train the last stage of ResNet50
for layer in base_model.layers[0:143]:
  layer.trainable = False

for layer in base_model.layers[143:]:
  layer.trainable = True

model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)
# Saving the weights in the current directory
model.save_weights("resnet50_weights.h5")


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/15
155/625 [======>.......................] - ETA: 2:44:22 - loss: 5.0845 - accuracy: 0.6605